# Define Audit Process

In [ ]:
%addconnect name=Transcend, user=sh186014, host=tdprdcop3.td.teradata.com, props="LOGMECH=LDAP"

In [ ]:
%connect name=Transcend

In [ ]:
select top 10 * 
from adlste_coa.coa_audit_summary 
order by 1 desc

In [ ]:
select top 10 * from adlste_coa.coa_audit_summary order by 1 desc

In [ ]:
call adlste_coa.sp_meta_objects_refresh('none');

In [ ]:
Select * from adlste_coa.coa_meta_objects
where Tablename  like '%mat%'
order by ObjType, ProcessName,  ObjType desc;

<br/><br/><br/>
## Audit_Run (sub-stored_procedure)
### for tracking audit columns per row while minimizing storage
- database:    adlste_coa
- table:       coat_audit_run
- view:        coa_audit_run
- stored proc: sp_audit_run

In [ ]:
drop table adlste_coa.coat_audit_runlog

In [11]:
rename table adlste_coa.coat_audit_runlog to adlste_coa.coat_audit_runlog_bkup

Success: 2 rows affected

In [12]:
Create multiset table adlste_coa.coat_audit_runlog
(
 Run_ID       INTEGER 
,ParentRun_ID INTEGER
,Site_ID      VARCHAR(128)
,TableName    VARCHAR(128)
,StartDate    DATE
,EndDate      DATE
,CallingSP    VARCHAR(128)
,SQLVersion   VARCHAR(128)
,Run_Type     VARCHAR(128)
,Run_Status   VARCHAR(128)
,Activity_Cnt BIGINT
,Run_Message  VARCHAR(512)
,Run_User     VARCHAR(128) Default USER
,Run_TS       Timestamp(0) Default Current_Timestamp(0)
) 
primary index (Site_ID, TableName);

Success: 0 rows affected

In [13]:
replace view adlste_coa.coa_audit_runlog 
as locking row for access
Select * from adlste_coa.coat_audit_runlog

Success: 14 rows affected

In [15]:
replace view adlste_coa.coa_audit_log
as locking row for access
Select * from adlste_coa.coat_audit_runlog

Success: 14 rows affected

In [14]:
replace PROCEDURE adlste_coa.sp_audit_log (inout runid integer
                                              ,in siteid varchar(128)
                                              ,in tblname varchar(128)
                                              ,in callingsp varchar(128)
                                              ,in sqlversion varchar(128)
                                              ,in startdate date
                                              ,in enddate date
                                              ,in runtype varchar(128)
                                              ,in runstatus varchar(128)
                                              ,in runmessage varchar(512)
                                              )
SQL SECURITY CREATOR
BEGIN
DECLARE runidout  INTEGER;
DECLARE runidnext INTEGER;

    SELECT coalesce(Max(Run_ID)+1,1) as Run_ID   INTO runidnext 
    FROM adlste_coa.coat_audit_runlog 
    ;

    IF runid = 0 THEN
        SET runidout = runidnext;
    ELSE
        SET runidout = runid;
    END IF;   
    
    Insert into adlste_coa.coat_audit_runlog 
    values
    ( :runidnext
     ,:runid --parentrunid
     ,:siteid
     ,:tblname
     ,:startdate
     ,:enddate
     ,:callingsp
     ,:sqlversion
     ,:runtype
     ,:runstatus
     ,:ACTIVITY_COUNT
     ,:runmessage
     ,USER 
     ,Current_Timestamp(0)
    );
    
    SET runid = runidout ;

END;

Success: 0 rows affected

In [16]:
insert into adlste_coa.coat_audit_runlog
Select
Run_ID       
,ParentRun_ID 
,Site_ID      
,"TableName"    
,StartDate    
,EndDate      
,CallingSP    
,SQLVersion   
,Run_Type     
,Run_Status   
,NULL as Activity_Cnt 
,Run_Message  
,Run_User     
,Run_TS       
from adlste_coa.coat_audit_runlog_bkup

Success: 2967 rows affected

In [ ]:
-- delete from adlste_coa.coat_audit_runlog -- don't do this

### Example of some calls - designed to be embedded in other SPs

In [ ]:
-- start SP()
CALL adlste_coa.sp_audit_log(0
                            ,'test_SiteID'  -- Site_ID
                            ,'adlste_coa.coat_audit_run'  -- Table Name
                            ,'adlste_coa.sp_test'  -- Calling SP
                            ,'v1.15'  -- StoredProc Version
                            ,DATE-7 -- Start Date
                            ,DATE-1 -- End Date
                            ,'Test Run'  -- Run Type
                            ,'START' );  -- Run Status

In [ ]:
-- do stuff, but need a checkpoint 
-- (number above becomes last parameter below)
call adlste_coa.sp_audit_log(4
                            ,'test_SiteID'  -- Site_ID
                            ,'adlste_coa.coat_audit_run'  -- Table Name
                            ,'adlste_coa.sp_test'  -- Calling SP
                            ,'v1.15'  -- StoredProc Version
                            ,DATE-7 -- Start Date
                            ,DATE-1 -- End Date
                            ,'Test Run'  -- Run Type
                            ,'CHECKPOINT' );  -- Run Status

In [ ]:
-- end of SP()
-- (number above becomes last parameter below)
call adlste_coa.sp_audit_log(4
                            ,'test_SiteID'  -- Site_ID
                            ,'adlste_coa.coat_audit_run'  -- Table Name
                            ,'adlste_coa.sp_test'  -- Calling SP
                            ,'v1.15'  -- StoredProc Version
                            ,DATE-7 -- Start Date
                            ,DATE-1 -- End Date
                            ,'Test Run'  -- Run Type
                            ,'END'); -- Run Status

In [ ]:
replace view adlste_coa.coa_audit_log as
locking row for access
Select * from adlste_coa.coat_audit_runlog

In [ ]:
replace view adlste_coa.coa_audit_summary   as locking row for access
Select 
 case when ParentRun_ID=0 then Run_ID else ParentRun_ID end as Run_ID
,Site_ID
,min(cast(Run_TS as DATE)) as LogDate
,TableName
,CallingSP
,SQLVersion
,Run_Type
,count(*) as Steps
,sum(case when Run_Status='ERROR' then 1 else 0 end) as Errors
,sum(case when Run_Status='WARNING' then 1 else 0 end) as Warnings
,min(case when Run_Status='START' then Run_TS else NULL end) as Start_TS
,max(case when Run_Status='END' then Run_TS else NULL end) as End_TS
from adlste_coa.coa_audit_runlog
Group by 1,2,4,5,6,7

In [ ]:
replace view adlste_coa.coa_audit_log_summary as
select * from adlste_coa.coa_audit_summary 

In [17]:
select top 100 * from adlste_coa.coa_audit_log order  by 1 desc

Run_ID,ParentRun_ID,Site_ID,TableName,StartDate,EndDate,CallingSP,SQLVersion,Run_Type,Run_Status,Activity_Cnt,Run_Message,Run_User,Run_TS
2973,2970,AAPROD4,adlste_coa.CB_DAILY_SUMMARY_DEPT_USER_DAY,2020-07-01,2020-08-30,adlste_coa.sp_dat_CB_DAILY_SUMMARY_DEPT_USER_DAY,0.6,normal run,END,,,GG185047,2020-10-28 09:48:10
2972,2970,AAPROD4,adlste_coa.CB_DAILY_SUMMARY_DEPT_USER_DAY,2020-07-01,2020-08-30,adlste_coa.sp_dat_CB_DAILY_SUMMARY_DEPT_USER_DAY,0.6,normal run,STEP,,Inserted stg into stg_CB_DAILY_SUMMARY_CURR_PREV,GG185047,2020-10-28 09:48:10
2971,2970,AAPROD4,adlste_coa.CB_DAILY_SUMMARY_DEPT_USER_DAY,2020-07-01,2020-08-30,adlste_coa.sp_dat_CB_DAILY_SUMMARY_DEPT_USER_DAY,0.6,normal run,STEP,,Removed existing records from CB_DAILY_SUMMARY_DEPT_USER_DAY,GG185047,2020-10-28 09:48:10
2970,0,AAPROD4,adlste_coa.CB_DAILY_SUMMARY_DEPT_USER_DAY,2020-07-01,2020-08-30,adlste_coa.sp_dat_CB_DAILY_SUMMARY_DEPT_USER_DAY,0.6,normal run,START,,,GG185047,2020-10-28 09:48:09
2969,2966,AAPROD4,adlste_coa.CB_DAILY_SUMMARY_DEPT_USER_HR,2020-07-31,2020-08-31,adlste_coa.sp_dat_CB_DAILY_SUMMARY_DEPT_USER_HR,0.6,normal run,END,,,GG185047,2020-10-28 09:45:19
2968,2966,AAPROD4,adlste_coa.CB_DAILY_SUMMARY_DEPT_USER_HR,2020-07-31,2020-08-31,adlste_coa.sp_dat_CB_DAILY_SUMMARY_DEPT_USER_HR,0.6,normal run,STEP,,Inserted stg into stg_CB_DAILY_SUMMARY_CURR_PREV_WK,GG185047,2020-10-28 09:45:19
2967,2966,AAPROD4,adlste_coa.CB_DAILY_SUMMARY_DEPT_USER_HR,2020-07-31,2020-08-31,adlste_coa.sp_dat_CB_DAILY_SUMMARY_DEPT_USER_HR,0.6,normal run,STEP,,Removed existing records from CB_DAILY_SUMMARY_DEPT_USER_HR,GG185047,2020-10-28 09:45:19
2966,0,AAPROD4,adlste_coa.CB_DAILY_SUMMARY_DEPT_USER_HR,2020-07-31,2020-08-31,adlste_coa.sp_dat_CB_DAILY_SUMMARY_DEPT_USER_HR,0.6,normal run,START,,,GG185047,2020-10-28 09:45:19
2965,2962,AAPROD4,adlste_coa.CB_DAILY_SUMMARY_CURR_PREV,2020-08-01,2020-08-31,adlste_coa.sp_dat_CB_DAILY_SUMMARY_CURR_PREV,0.6,normal run,END,,,GG185047,2020-10-28 09:44:39
2964,2962,AAPROD4,adlste_coa.CB_DAILY_SUMMARY_CURR_PREV,2020-08-01,2020-08-31,adlste_coa.sp_dat_CB_DAILY_SUMMARY_CURR_PREV,0.6,normal run,STEP,,Inserted stg into stg_CB_DAILY_SUMMARY_CURR_PREV_WK,GG185047,2020-10-28 09:44:39


In [ ]:
select * from adlste_coa.coa_audit_summary order  by 1 desc